## Segmentation

In [11]:
merge_dataset_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun - Use/merged.csv"

import pandas as pd

merge_df = pd.read_csv(merge_dataset_file_path)
merge_df.head()


,customerid,city,state,totalOrders,totalSpend,avgOrderValue,lastPurchaseDate,firstPurchaseDate,daysSinceLastPurchase,customerLifetimeMonths,purchaseFrequency,favoritePaymentMethod,favoriteItem,favoritePurchaseHour,favoriteDayPart,recency,frequency,monetary
0,CUST0001,Sungai Besi,Wilayah Persekutuan Kuala Lumpur,2,65.0,32.5,03/11/2021,03/11/2021,1475.0,0,2,Unknown,Oats & Calendula Dandruff Shampoo Bar | Sulfat...,20.0,Evening,1475.0,2,65.0
1,CUST0002,Mutiara Damansara,Selangor,2,94.0,47.0,04/11/2021,04/11/2021,1474.0,0,2,Unknown,Green Tea & Mint Hair Loss Shampoo | Sulfate-F...,15.0,Afternoon,1474.0,2,94.0
2,CUST0003,Shah Alam,Selangor,1,26.0,26.0,04/11/2021,04/11/2021,1474.0,0,1,E-Wallet,Green Tea & Mint Hair Loss Shampoo | Sulfate-F...,16.0,Afternoon,1474.0,1,26.0
3,CUST0004,Muar,Johor,3,369.0,123.0,04/11/2021,04/11/2021,1474.0,0,3,Unknown,Green Tea & Mint Hair Loss Shampoo | Sulfate-F...,21.0,Evening,1474.0,3,369.0
4,CUST0005,Petaling Jaya,Selangor,1,25.0,25.0,05/11/2021,05/11/2021,1473.0,0,1,Card,Green Tea & Mint Hair Loss Shampoo | Sulfate-F...,9.0,Morning,1473.0,1,25.0


In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.pipeline import Pipeline

import json
import warnings
warnings.filterwarnings("ignore")


# ====================================================
#  MAIN SEGMENTATION PIPELINE
#  Input: df (cleaned dataframe), selected_features (list from frontend)
# ====================================================

def run_segmentation(df, selected_features):
    """
    Full customer segmentation pipeline:
    - Extract only needed features
    - Encode categorical
    - Scale numeric
    - Auto-select K (2–10)
    - Evaluate using silhouette + DBI
    - Run final KMeans
    - Generate summary
    - Return Python dict (Node will send as JSON)
    """

    # --- 1. Select subset for clustering ---
    usable_df = df[selected_features].copy()

    # Identify categorical vs numeric
    categorical_cols = usable_df.select_dtypes(include=['object']).columns.tolist()
    numeric_cols = usable_df.select_dtypes(include=[np.number]).columns.tolist()

    # --- 2. Build preprocessing pipeline ---
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
            ("num", StandardScaler(), numeric_cols)
        ]
    )
    
    X_transformed = preprocessor.fit_transform(usable_df)
    
    # If one-hot creates sparse matrix → convert to dense
    if hasattr(X_transformed, "toarray"):
        X_transformed = X_transformed.toarray()
    
    # --- LOGGING ---
    print("=== Panel Demo: Encoding & Scaling ===")
    if categorical_cols:
        print("[LOG - ONEHOTENCODE] Original categorical data:")
        print(usable_df[categorical_cols].head())
        print("After OneHotEncoding (first 5 rows of transformed data):")
        # Extract encoded part
        cat_encoded_len = sum([len(preprocessor.named_transformers_['cat'].categories_[i]) for i in range(len(categorical_cols))])
        print(X_transformed[:, :cat_encoded_len][:5, :])
        print("Feature names:", preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols))
        print("-----------------------------------\n")

    if numeric_cols:
        print("[LOG - SCALE] Original numeric data:")
        print(usable_df[numeric_cols].head())
        print("After Scaling (first 5 rows of transformed data):")
        print(X_transformed[:, -len(numeric_cols):][:5, :])
        print("-----------------------------------\n")

    print("Combined transformed data (first 5 rows):")
    print(X_transformed[:5, :])
    print("Transformed shape:", X_transformed.shape)
    print("=====================================\n")

    # --- 3. Test K from 2 to 10 ---
    k_results = []
    best_k = None
    best_score = -1

    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k, random_state=42)

        pipeline = Pipeline([
            ("preprocess", preprocessor),
            ("cluster", kmeans)
        ])

        labels = pipeline.fit_predict(usable_df)

        # Convert sparse matrix to dense array for metrics
        X_transformed = pipeline.named_steps["preprocess"].transform(usable_df)
        if hasattr(X_transformed, 'toarray'):
            X_transformed = X_transformed.toarray()
        
        silhouette = silhouette_score(X_transformed, labels)
        dbi = davies_bouldin_score(X_transformed, labels)

        k_results.append({
            "k": k,
            "silhouette": float(silhouette),
            "dbi": float(dbi)
        })

        if silhouette > best_score:
            best_score = silhouette
            best_k = k

    # --- 4. Final KMeans using best K ---
    final_kmeans = KMeans(n_clusters=best_k, random_state=42)
    final_pipeline = Pipeline([
        ("preprocess", preprocessor),
        ("cluster", final_kmeans)
    ])

    final_labels = final_pipeline.fit_predict(usable_df)

    # Attach cluster labels to df
    df["cluster"] = final_labels

    # --- 5. Generate segment profiles ---
    summary = generate_cluster_summary(df, selected_features)

    # --- 6. Full response ---
    response = {
        "best_k": int(best_k),
        "evaluation": k_results,
        "cluster_assignments": df[["customerid", "cluster"]].to_dict(orient="records"),
        "cluster_summary": summary
    }

    return response


# ====================================================
#  CLUSTER SUMMARY GENERATION
# ====================================================

def generate_cluster_summary(df, selected_features):

    clusters = df["cluster"].unique()
    summary = {}

    for c in clusters:
        cluster_df = df[df["cluster"] == c]

        stats = {}

        for col in selected_features:
            if cluster_df[col].dtype == "O":  # categorical
                top = cluster_df[col].value_counts().idxmax()
                stats[col] = str(top)
            else:  # numeric
                stats[col] = float(cluster_df[col].mean())

        summary[f"cluster_{c}"] = {
            "count": int(len(cluster_df)),
            "percentage": round((len(cluster_df) / len(df) * 100), 2),
            "attributes": stats
        }

    return summary



# ====================================================
# EXAMPLE usage (you can remove this in production)
# ====================================================
if __name__ == "__main__":
    df = pd.read_csv(merge_dataset_file_path)

    selected = ["state", "totalSpend"]   # example
    result = run_segmentation(df, selected)

    print(json.dumps(result, indent=4))


=== Panel Demo: Encoding & Scaling ===
[LOG - ONEHOTENCODE] Original categorical data:
                              state
0  Wilayah Persekutuan Kuala Lumpur
1                          Selangor
2                          Selangor
3                             Johor
4                          Selangor
After OneHotEncoding (first 5 rows of transformed data):
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
Feature names: ['state_Johor' 'state_Kedah' 'state_Kelantan' 'state_Melaka'
 'state_Negeri Sembilan' 'state_Pahang' 'state_Perak' 'state_Perlis'
 'state_Pulau Pinang' 'state_Sabah' 'state_Sarawak' 'state_Selangor'
 'state_Terengganu' 'state_Wilayah Persekutuan Kuala Lumpur'
 'state_Wilayah Persekutuan Labuan' 'state_Wilayah Persekutuan Putrajaya']
----------------------------------

: 